# 📊 AWS Glue & PySpark ETL: Análise de Transações Financeiras

## 📌 Introdução

Este projeto tem como objetivo **analisar transações financeiras**, identificando padrões e possíveis fraudes.
Utilizamos **AWS Glue & PySpark** para processamento e **Pandas + Seaborn** para análise exploratória.

### **Principais perguntas analisadas:**
- Como os valores das transações estão distribuídos?
- Há padrões claros entre transações fraudulentas e legítimas?
- Existe alguma variável fortemente correlacionada com fraudes?

### **Tecnologias utilizadas:**
✅ **PySpark & Pandas** → Manipulação de dados.  
✅ **Matplotlib & Seaborn** → Visualização de insights.  
✅ **AWS Glue & S3** → Processamento em escala.  
✅ **SQL & Power BI** → Otimização e Dashboards (próximas etapas).


## 📌 2️⃣ Importação de Bibliotecas

In [ ]:
import os
import yaml
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## 📌 3️⃣ Carregamento do Arquivo de Configuração

Antes de iniciar a análise, precisamos carregar as configurações do projeto.  
O arquivo `config.yaml` contém informações essenciais, como **caminho dos dados** e **parâmetros de processamento**.


In [ ]:
import os
import yaml

# 📂 Garantir que o caminho seja carregado corretamente
config_path = os.path.abspath(os.path.join(os.getcwd(), "config/config.yaml"))

print(f"📂 Tentando carregar: {config_path}")

if os.path.exists(config_path):
    with open(config_path, "r") as f:
        config = yaml.safe_load(f)
    print("✅ Configuração carregada com sucesso!")
else:
    raise FileNotFoundError("❌ Arquivo 'config.yaml' não encontrado!")

# 🔄 Definir ambiente
IS_AWS = config.get("environment") == "aws"

# 📂 Ajustar caminhos dependendo do ambiente
if IS_AWS:
    INPUT_PATH = config.get("aws_s3_input")
    OUTPUT_PATH = config.get("aws_s3_output")
else:
    INPUT_PATH = os.path.abspath(config.get("raw_data_path"))
    OUTPUT_PATH = os.path.abspath(config.get("data_path"))

print(f"📂 Caminho de entrada: {INPUT_PATH}")
print(f"📂 Caminho de saída: {OUTPUT_PATH}")


In [ ]:
# Configuração para exibição de gráficos inline
%matplotlib inline
sns.set_theme(style="whitegrid")

# 📌 4️⃣ Carregar os Dados

In [ ]:
import os
import yaml
import glob
import pandas as pd

# 📂 Carregar configuração do YAML
config_path = os.path.abspath("config/config.yaml")

if os.path.exists(config_path):
    with open(config_path, "r") as f:
        config = yaml.safe_load(f)
    print("✅ Configuração carregada com sucesso!")
else:
    raise FileNotFoundError("❌ Arquivo 'config.yaml' não encontrado!")

# 🔄 Definir ambiente (AWS ou Local)
IS_AWS = config.get("environment") == "aws"

# 📂 Ajustar caminho de busca dos arquivos Parquet
if IS_AWS:
    processed_data_path = config.get("aws_s3_output")  # No AWS, leitura seria via S3
else:
    processed_data_path = os.path.abspath(config.get("data_path"))

# 📂 Buscar arquivos Parquet no diretório configurado
parquet_files = glob.glob(os.path.join(processed_data_path, "*.parquet"))

# 🔄 Corrigir caminho para evitar problemas no Windows/Linux
parquet_files = [os.path.normpath(f) for f in parquet_files]

# 📌 Verificar se encontrou arquivos Parquet
if not parquet_files:
    raise FileNotFoundError(f"❌ Nenhum arquivo Parquet encontrado no diretório: {processed_data_path}")

print(f"✅ {len(parquet_files)} arquivos Parquet encontrados!")

# 🔄 Carregar e concatenar os arquivos Parquet no Pandas DataFrame
df_list = [pd.read_parquet(f, engine="pyarrow") for f in parquet_files]
df = pd.concat(df_list, ignore_index=True) if df_list else pd.DataFrame()

# 📊 Exibir primeiras linhas e estatísticas
display(df.head())
display(df.describe())
display(df.dtypes)


# 📌 5️⃣ Exploração Inicial dos Dados

In [ ]:
print("📊 Exibindo as 5 primeiras linhas do dataset:")
display(df.head())

print("\n📌 Informações Gerais sobre o DataFrame:")
df.info()

print("\n📊 Estatísticas descritivas do dataset:")
display(df.describe())

print("\n🔍 Verificação de valores nulos:")
display(df.isnull().sum())

print("\n📌 Tipos de dados das colunas:")
display(df.dtypes)


## 📌 6️⃣ Análise de Distribuição dos Valores das Transações

Nesta seção, analisamos **como os valores das transações estão distribuídos** e **se há outliers**.  
Essa análise nos ajudará a entender padrões e possíveis anomalias associadas a fraudes.


### 📊 6.1 Histograma: Distribuição do Valor das Transações
- **Objetivo**: Visualizar a distribuição dos valores das transações.  
- **Interpretação**:  
  - A maioria das transações tem valores **baixos (0 a 10 dólares)**.  
  - A distribuição é **assimétrica à direita**, indicando algumas transações de alto valor.  
  - Transações de valores extremos podem ser **outliers ou potenciais fraudes**.


In [ ]:
plt.figure(figsize=(10,5))
sns.histplot(df["amt"], bins=50, kde=True)
plt.title("Distribuição do Valor das Transações")
plt.xlabel("Valor da Transação ($)")
plt.ylabel("Frequência")
plt.show()


###  6.2 Boxplot: Identificação de Outliers
- **Objetivo**: Identificar **outliers (valores atípicos)** nas transações.  
- **Interpretação**:  
  - A mediana (~50 dólares) representa o valor central da maioria das transações.  
  - **Valores acima de ~175 dólares são considerados outliers**, indicando transações incomuns.  
  - Transações extremamente altas podem ser **fraudes ou compras legítimas de alto valor**.


In [ ]:
plt.figure(figsize=(10,5))
sns.boxplot(x=df["amt"])
plt.title("Boxplot do Valor das Transações")
plt.xlabel("Valor da Transação ($)")
plt.show()


### 6.3 Filtrando Transações com Valores Elevados
- **Objetivo**: Examinar transações acima de **175 dólares** para verificar se há padrões incomuns.  
- **Interpretação**:  
  - Essas transações podem ser **raras e merecem investigação**.  
  - Se muitas dessas transações forem fraudulentas (`is_fraud = 1`), pode indicar que fraudes ocorrem **principalmente em valores altos**.


In [ ]:
high_value_tx = df[df["amt"] > 175]
display(high_value_tx.head())


### 📊 6.4 Percentual de Transações Elevadas que São Fraudes
- **Objetivo**: Contar quantas dessas transações de alto valor são fraudulentas (`is_fraud = 1`).  
- **Interpretação**:  
  - Se um **percentual significativo** de fraudes estiver entre as transações caras, pode indicar um **padrão de fraude em compras de alto valor**.  
  - Se a maioria das transações de alto valor for legítima, pode ser apenas um comportamento esperado dos clientes.


In [ ]:
df[df["amt"] > 175]["is_fraud"].value_counts()


## 📌 7️⃣ Matriz de Correlação

"""
A correlação entre as variáveis nos ajuda a entender **relações estatísticas** entre elas.
Valores próximos de **1.0 ou -1.0** indicam **forte correlação positiva ou negativa**, 
enquanto valores próximos de **0** indicam que as variáveis são independentes.

📊 **O que analisar?**
- Se o valor da transação (`amt`) tem correlação com fraude (`is_fraud`).
- Se alguma variável de localização (`lat`, `long`) pode indicar comportamento suspeito.
- Se o tempo entre transações (`time_diff`) influencia fraudes.
"""

In [ ]:
# Selecionar apenas colunas numéricas para evitar erro
df_numeric = df.select_dtypes(include=['number'])

# Criar matriz de correlação
plt.figure(figsize=(10,6))
sns.heatmap(df_numeric.corr(), annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
plt.title("Matriz de Correlação das Variáveis")
plt.show()

In [ ]:
## 📌 8️⃣ Conclusões e Próximos Passos

### 🔍 **Conclusões**
1️⃣ O dataset contém **informações completas e bem estruturadas**.  
2️⃣ A maioria das transações está **concentrada em valores menores**, com poucos outliers.  
3️⃣ **Fraudes podem ocorrer tanto em valores baixos quanto altos**, indicando que **outros fatores devem ser analisados**.

### 🚀 **Próximos Passos**
✅ **Testar otimizações de performance no PySpark** para melhorar a eficiência do ETL.  
✅ **Criar dashboards interativos no Power BI** para monitoramento em tempo real.
